# RL asmt 1

Discuss in detail how Markov Decision Process (MDP) can help in route planning application.
Justify your answer in 350-400 words. Explain in detail your environment, state space, action
space  and equations aligned with the given problem. Write the answer in the Colab Cell itself.


In [1]:
import pandas as pd
import numpy as np

In [81]:
# # Reading the df and finding the prior, likelihood, predictor prior and posterior

tmp = [[1, 'Best',    'Minimum', 'Medium', 'Yes'],
       [2, 'Average', 'Minimum', 'Medium', 'Yes'],
       [3, 'Best',    'Maximum', 'Low',    'No'],
       [6, 'Worst',   'Maximum', 'High',   'Yes'],
       [4, 'Average', 'Minimum', 'High',   'No'],
       [1, 'Average', 'Minimum', 'Low',    'Yes'],
       [5, 'Average', 'Maximum', 'High',   'Yes'],
       [7, 'Worst',   'Minimum', 'High',   'Yes'],
       [8, 'Best',    'Minimum', 'Medium', 'No'],
       [2, 'Best',    'Maximum', 'Medium', 'Yes'],
       [3, 'Worst',   'Maximum', 'Medium', 'No'],
       [8, 'Best',    'Minimum', 'Low',    'Yes'],
       [5, 'Worst',   'Maximum', 'High',   'No'],
       [4, 'Worst',   'Minimum', 'Medium', 'No'],
       [3, 'Best',    'Maximum', 'Low',    'Yes'],
       [7, 'Average', 'Minimum', 'Low',    'Yes'],
       [6, 'Average', 'Minimum', 'Low',    'Yes'],
       [1, 'Average', 'Maximum', 'Low',    'No']]

df = pd.DataFrame(tmp, columns = ['Route', 'Time', 'Cost', 'Traffic Intensity', 'Selection'])

df.head(20)

,Route,Time,Cost,Traffic Intensity,Selection
0,1,Best,Minimum,Medium,Yes
1,2,Average,Minimum,Medium,Yes
2,3,Best,Maximum,Low,No
3,6,Worst,Maximum,High,Yes
4,4,Average,Minimum,High,No
5,1,Average,Minimum,Low,Yes
6,5,Average,Maximum,High,Yes
7,7,Worst,Minimum,High,Yes
8,8,Best,Minimum,Medium,No
9,2,Best,Maximum,Medium,Yes


In [82]:
prior = df['Selection'].value_counts(normalize=True).round(6)  # class prior
predictor_prior = df['Route'].value_counts(normalize=True).round(6) # predictor prior
likelihood_no = df[df['Selection']=='No']['Route'].value_counts(normalize=True).round(6).sort_index() # likelihood
likelihood_yes = df[df['Selection']=='Yes']['Route'].value_counts(normalize=True).round(6).sort_index()


In [83]:
prior

Selection
Yes    0.611111
No     0.388889
Name: proportion, dtype: float64

In [84]:
predictor_prior

Route
1    0.166667
3    0.166667
2    0.111111
6    0.111111
4    0.111111
5    0.111111
7    0.111111
8    0.111111
Name: proportion, dtype: float64

In [85]:
likelihood_yes

Route
1    0.181818
2    0.181818
3    0.090909
5    0.090909
6    0.181818
7    0.181818
8    0.090909
Name: proportion, dtype: float64

In [86]:
def find_posterior(class_likelihood, class_prior, pred_prior):
    tmp = class_likelihood * class_prior  # class posterior = class likelihood * class prior / predictor prior
    aligned_series1, aligned_series2 = tmp.align(pred_prior)
    return (aligned_series1/aligned_series2).round(6)

def normalise_posterior(a, b):
    sum_series = a + b
    return a/sum_series, b/sum_series

posterior_yes_ = find_posterior(likelihood_yes , prior['Yes'], predictor_prior).fillna(1)
posterior_no_ = find_posterior(likelihood_no , prior['No'], predictor_prior).fillna(1)

posterior_yes, posterior_no = normalise_posterior(posterior_yes_, posterior_no_)
posterior_yes

Route
1    0.666667
2    0.500000
3    0.333333
4    0.500000
5    0.500000
6    0.500000
7    0.500000
8    0.500000
Name: proportion, dtype: float64

In [87]:
posterior_no

Route
1    0.333333
2    0.500000
3    0.666667
4    0.500000
5    0.500000
6    0.500000
7    0.500000
8    0.500000
Name: proportion, dtype: float64

State transition probability P(St+1=S1 | St = S0, At=a).
In our case can be written as P(St+1=S1 | St = S0)  [State x State Matrix]

In [88]:
# P(Selection=yes|R1)/( P(yes|R1) + P(yes|R2)+ P(yes|R3) ) for state A to C.
Pca_ = (posterior_yes[1]/(posterior_yes[1]+posterior_yes[2]+posterior_yes[3])).round(6)  # normalisation
Pda_ = (posterior_yes[3]/(posterior_yes[1]+posterior_yes[2]+posterior_yes[3])).round(6)
Pfa_ = (posterior_yes[2]/(posterior_yes[1]+posterior_yes[2]+posterior_yes[3])).round(6)

Pcf_ = (posterior_yes[4]/(posterior_yes[4]+posterior_yes[6])).round(6)
Pef_ = (posterior_yes[6]/(posterior_yes[4]+posterior_yes[6])).round(6)

state_transition_matrix = {
    'A' : {'A':0, 'B':0, 'C':Pca_, 'D':Pda_, 'E':0, 'F':Pfa_ },  # keys - initial, dict entries final
    'B' : {'A':0, 'B':0, 'C':0, 'D':0, 'E':0, 'F':0 },
    'C' : {'A':0, 'B':1, 'C':0, 'D':0, 'E':0, 'F':0 },
    'D' : {'A':0, 'B':0, 'C':0, 'D':0, 'E':1, 'F':0 },
    'E' : {'A':0, 'B':1, 'C':0, 'D':0, 'E':0, 'F':0 },
    'F' : {'A':0, 'B':0, 'C':Pcf_, 'D':0, 'E':Pef_, 'F':0 }
}

state_transition_matrix  # probability of transitioning to state x from state y

{'A': {'A': 0, 'B': 0, 'C': 0.444445, 'D': 0.222222, 'E': 0, 'F': 0.333333},
 'B': {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0},
 'C': {'A': 0, 'B': 1, 'C': 0, 'D': 0, 'E': 0, 'F': 0},
 'D': {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 1, 'F': 0},
 'E': {'A': 0, 'B': 1, 'C': 0, 'D': 0, 'E': 0, 'F': 0},
 'F': {'A': 0, 'B': 0, 'C': 0.5, 'D': 0, 'E': 0.5, 'F': 0}}

Wherever there is only one path, we pick it so transition probability is 1.
Where there are multiple paths, we use normalised P(yes|route) to find state transition probabilities

In [89]:
# also keeping a mapping of inital_state->actions->final_state
state_action_mapping = {'A':{'R1':'C', 'R2':'F', 'R3':'D'},
                        'B':{},
                        'C':{'R7':'B'},
                        'D':{'R5':'E'},
                        'E':{'R8':'B'},
                        'F':{'R4':'C', 'R6':'E'}}

state_action_mapping

{'A': {'R1': 'C', 'R2': 'F', 'R3': 'D'},
 'B': {},
 'C': {'R7': 'B'},
 'D': {'R5': 'E'},
 'E': {'R8': 'B'},
 'F': {'R4': 'C', 'R6': 'E'}}

We will use E[Cost|RouteA] as Reward. Here cost can be switched with time taken or traffic or weighted average of these

In [90]:
time_mapping = {'Best':5,'Average':3,'Worst':1}
cost_mapping = {'Minimum':4, 'Maximum':1}
traffic_mapping = {'Low':5, 'Medium':4,'High':1}

df['Time'] = df['Time'].replace(time_mapping)
df['Cost'] = df['Cost'].replace(cost_mapping)
df['Traffic Intensity'] = df['Traffic Intensity'].replace(traffic_mapping)

df.head(15)

,Route,Time,Cost,Traffic Intensity,Selection
0,1,5,4,4,Yes
1,2,3,4,4,Yes
2,3,5,1,5,No
3,6,1,1,1,Yes
4,4,3,4,1,No
5,1,3,4,5,Yes
6,5,3,1,1,Yes
7,7,1,4,1,Yes
8,8,5,4,4,No
9,2,5,1,4,Yes


In [91]:
rewards = df.groupby('Route').agg({'Time':'mean', 'Cost':'mean', 'Traffic Intensity':'mean'})
reward_time = rewards['Time'].round(6)  # expected time based reward if action A is chosen
reward_cost = rewards['Cost'].round(6)  # same for traffic and cost
reward_traffic = rewards['Traffic Intensity'].round(6)
reward_combined =  (0.33*reward_time + 0.33*reward_cost + 0.33*reward_traffic).round(6)
# considering all factors with equal weightage

reward_time

Route
1    3.666667
2    4.000000
3    3.666667
4    2.000000
5    2.000000
6    2.000000
7    2.000000
8    5.000000
Name: Time, dtype: float64

In [92]:
reward_cost

Route
1    3.0
2    2.5
3    1.0
4    4.0
5    1.0
6    2.5
7    4.0
8    4.0
Name: Cost, dtype: float64

In [93]:
reward_traffic

Route
1    4.666667
2    4.000000
3    4.666667
4    2.500000
5    1.000000
6    3.000000
7    3.000000
8    4.500000
Name: Traffic Intensity, dtype: float64

In [94]:
reward_combined

Route
1    3.740
2    3.465
3    3.080
4    2.805
5    1.320
6    2.475
7    2.970
8    4.455
dtype: float64

In [95]:
reward_dummy = reward_traffic.copy()  # taking a dummy -1 per step reward to solve minimum step problem
reward_dummy[:]=-1
reward_dummy

Route
1   -1.0
2   -1.0
3   -1.0
4   -1.0
5   -1.0
6   -1.0
7   -1.0
8   -1.0
Name: Traffic Intensity, dtype: float64

In [126]:
# Running value iteration script - time
def value_iteration(reward_type):
    num_iterations = 0
    value_vector = pd.Series({'A':50, 'B':100, 'C':50, 'D':50, 'E':50, 'F':50 }, name='value') # starting with an inital value estimate
    gamma = 1 # setting up a discounting factor
    while num_iterations < 5:
        updated_value = {}
        for state in ['A', 'B', 'C', 'D', 'E', 'F']:
            new_state_value = -10**12
            for action in range(1, 9):
                if 'R'+str(action) in list(state_action_mapping[state].keys()):
                    # action belongs to the state, find next state
                    next_state = state_action_mapping[state]['R'+str(action)]
                    next_state_transition_probability = state_transition_matrix[state][next_state]

                    new_state_value = max(new_state_value,
                                          reward_type[action] + gamma * next_state_transition_probability * value_vector[next_state] )
                    print(' for iteration {}, state {} and action {}, next state transition prob {} '.format(
                        num_iterations, state, 'R'+str(action), next_state_transition_probability
                    ) )
                else:
                    # this action dosent belongs to this state
                    pass

            #v_k+1 = max(R_a + gamma* P_a * v_k)
            updated_value[state] = value_vector[state] if new_state_value == -10**12 else new_state_value

        updated_value_vector = pd.Series(updated_value, name='value')
        print('value vector at iteration {} is {}'.format(num_iterations, list(updated_value_vector)))
        value_vector = updated_value_vector

        num_iterations +=1
    return value_vector


final_value_vector = value_iteration(reward_type=reward_dummy)
final_value_vector

 for iteration 0, state A and action R1, next state transition prob 0.444445 
 for iteration 0, state A and action R2, next state transition prob 0.333333 
 for iteration 0, state A and action R3, next state transition prob 0.222222 
 for iteration 0, state C and action R7, next state transition prob 1 
 for iteration 0, state D and action R5, next state transition prob 1 
 for iteration 0, state E and action R8, next state transition prob 1 
 for iteration 0, state F and action R4, next state transition prob 0.5 
 for iteration 0, state F and action R6, next state transition prob 0.5 
value vector at iteration 0 is [21.22225, 100.0, 99.0, 49.0, 99.0, 24.0]
 for iteration 1, state A and action R1, next state transition prob 0.444445 
 for iteration 1, state A and action R2, next state transition prob 0.333333 
 for iteration 1, state A and action R3, next state transition prob 0.222222 
 for iteration 1, state C and action R7, next state transition prob 1 
 for iteration 1, state D and

A     43.000055
B    100.000000
C     99.000000
D     98.000000
E     99.000000
F     48.500000
Name: value, dtype: float64

In [127]:
final_value_vector = value_iteration(reward_type=reward_time)
final_value_vector

 for iteration 0, state A and action R1, next state transition prob 0.444445 
 for iteration 0, state A and action R2, next state transition prob 0.333333 
 for iteration 0, state A and action R3, next state transition prob 0.222222 
 for iteration 0, state C and action R7, next state transition prob 1 
 for iteration 0, state D and action R5, next state transition prob 1 
 for iteration 0, state E and action R8, next state transition prob 1 
 for iteration 0, state F and action R4, next state transition prob 0.5 
 for iteration 0, state F and action R6, next state transition prob 0.5 
value vector at iteration 0 is [25.888917, 100.0, 102.0, 52.0, 105.0, 27.0]
 for iteration 1, state A and action R1, next state transition prob 0.444445 
 for iteration 1, state A and action R2, next state transition prob 0.333333 
 for iteration 1, state A and action R3, next state transition prob 0.222222 
 for iteration 1, state C and action R7, next state transition prob 1 
 for iteration 1, state D 

A     49.000057
B    100.000000
C    102.000000
D    107.000000
E    105.000000
F     54.500000
Name: value, dtype: float64

In [128]:
final_value_vector = value_iteration(reward_type=reward_cost)
final_value_vector

 for iteration 0, state A and action R1, next state transition prob 0.444445 
 for iteration 0, state A and action R2, next state transition prob 0.333333 
 for iteration 0, state A and action R3, next state transition prob 0.222222 
 for iteration 0, state C and action R7, next state transition prob 1 
 for iteration 0, state D and action R5, next state transition prob 1 
 for iteration 0, state E and action R8, next state transition prob 1 
 for iteration 0, state F and action R4, next state transition prob 0.5 
 for iteration 0, state F and action R6, next state transition prob 0.5 
value vector at iteration 0 is [25.22225, 100.0, 104.0, 51.0, 104.0, 29.0]
 for iteration 1, state A and action R1, next state transition prob 0.444445 
 for iteration 1, state A and action R2, next state transition prob 0.333333 
 for iteration 1, state A and action R3, next state transition prob 0.222222 
 for iteration 1, state C and action R7, next state transition prob 1 
 for iteration 1, state D a

A     49.22228
B    100.00000
C    104.00000
D    105.00000
E    104.00000
F     56.00000
Name: value, dtype: float64

In [129]:
final_value_vector = value_iteration(reward_type=reward_traffic)
final_value_vector

 for iteration 0, state A and action R1, next state transition prob 0.444445 
 for iteration 0, state A and action R2, next state transition prob 0.333333 
 for iteration 0, state A and action R3, next state transition prob 0.222222 
 for iteration 0, state C and action R7, next state transition prob 1 
 for iteration 0, state D and action R5, next state transition prob 1 
 for iteration 0, state E and action R8, next state transition prob 1 
 for iteration 0, state F and action R4, next state transition prob 0.5 
 for iteration 0, state F and action R6, next state transition prob 0.5 
value vector at iteration 0 is [26.888917, 100.0, 103.0, 51.0, 104.5, 28.0]
 for iteration 1, state A and action R1, next state transition prob 0.444445 
 for iteration 1, state A and action R2, next state transition prob 0.333333 
 for iteration 1, state A and action R3, next state transition prob 0.222222 
 for iteration 1, state C and action R7, next state transition prob 1 
 for iteration 1, state D 

A     50.444502
B    100.000000
C    103.000000
D    105.500000
E    104.500000
F     55.250000
Name: value, dtype: float64

In [130]:
final_value_vector = value_iteration(reward_type=reward_combined)
final_value_vector

 for iteration 0, state A and action R1, next state transition prob 0.444445 
 for iteration 0, state A and action R2, next state transition prob 0.333333 
 for iteration 0, state A and action R3, next state transition prob 0.222222 
 for iteration 0, state C and action R7, next state transition prob 1 
 for iteration 0, state D and action R5, next state transition prob 1 
 for iteration 0, state E and action R8, next state transition prob 1 
 for iteration 0, state F and action R4, next state transition prob 0.5 
 for iteration 0, state F and action R6, next state transition prob 0.5 
value vector at iteration 0 is [25.962249999999997, 100.0, 102.97, 51.32, 104.455, 27.805]
 for iteration 1, state A and action R1, next state transition prob 0.444445 
 for iteration 1, state A and action R2, next state transition prob 0.333333 
 for iteration 1, state A and action R3, next state transition prob 0.222222 
 for iteration 1, state C and action R7, next state transition prob 1 
 for iterat

A     49.504502
B    100.000000
C    102.970000
D    105.775000
E    104.455000
F     54.702500
Name: value, dtype: float64